In [5]:
import artm
print artm.version()

0.8.1


In [80]:
batch_vectorizer = artm.BatchVectorizer(data_path='lurk_parsed_full_total_sort_3plus.vv',
                                        data_format='vowpal_wabbit',
                                        target_folder='lurk_batches_full_3plus')

print 'Done'

Done


In [81]:
batch_vectorizer.dictionary.save_text(dictionary_path='lurk_batches_full_3plus/lurk_dictionary.txt')

# Обучение LDA

In [82]:
lda = artm.LDA(num_topics=1000, num_document_passes=30, dictionary=batch_vectorizer.dictionary)

lda.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=20)

print lda.sparsity_phi_last_value
print lda.sparsity_theta_last_value
print lda.perplexity_value

# 200 [51710.749651565464, 2912.4517844033107, 1463.3352772163084, 1068.9170659403558, 940.4391144676547, 884.1208358691122, 855.6548172423146, 839.2603788026935, 828.7610683372303, 821.5051712358239]
# 300 [51630.84984424069, 2771.2873934652525, 1158.3322936601703, 792.455869809864, 695.4067568619424, 657.0467052353292, 638.7229803614259, 628.7030068003055, 622.0987749425461, 617.1389140891057]
# 500 [51504.93969941082, 2880.385673853591, 1089.606241698333, 721.7839106509626, 634.6241138052483, 600.9330195668217, 583.5704540795377, 573.105832796926, 566.3551174650534, 561.6759636252757]
# 600 518.4746326073434
# 1000 421
# 1000 403 (20 iter, 30 docs iter)
# 1500 372
# 3000 328
# 4000 324

0.0
0.0
[50600.87996657025, 2424.454962189418, 729.9491102869564, 512.446585788721, 459.0184753449112, 437.6998704794825, 427.24801946049126, 421.6334465023198, 418.0529976597233, 415.3252393485439, 413.1043897562646, 411.3671989502545, 409.8640517123054, 408.5292795744897, 407.4243205216838, 406.3941112488756, 405.41887678992, 404.66157727657213, 403.9962361569887, 403.375940465323]


# Результаты, темы:

In [87]:
import operator
hist = {}

topics = open('topics.txt', 'w')

top_tokens = lda.get_top_tokens(num_tokens=10)
for i, token_list in enumerate(top_tokens):
    print 'Topic {}: '.format(i),
    for t in sorted(token_list):
        hist[t] = hist.pop(t, 0) + 1
        topics.write(t.encode('utf-8') + ' ')
#         print t,
    topics.write('\n')
#     print '\n'
    
for t in sorted(hist.items(), key=operator.itemgetter(1), reverse=True)[:20]:
    print '  ', t[0], ' - ', t[1]

topics.close()

 Topic 0:  Topic 1:  Topic 2:  Topic 3:  Topic 4:  Topic 5:  Topic 6:  Topic 7:  Topic 8:  Topic 9:  Topic 10:  Topic 11:  Topic 12:  Topic 13:  Topic 14:  Topic 15:  Topic 16:  Topic 17:  Topic 18:  Topic 19:  Topic 20:  Topic 21:  Topic 22:  Topic 23:  Topic 24:  Topic 25:  Topic 26:  Topic 27:  Topic 28:  Topic 29:  Topic 30:  Topic 31:  Topic 32:  Topic 33:  Topic 34:  Topic 35:  Topic 36:  Topic 37:  Topic 38:  Topic 39:  Topic 40:  Topic 41:  Topic 42:  Topic 43:  Topic 44:  Topic 45:  Topic 46:  Topic 47:  Topic 48:  Topic 49:  Topic 50:  Topic 51:  Topic 52:  Topic 53:  Topic 54:  Topic 55:  Topic 56:  Topic 57:  Topic 58:  Topic 59:  Topic 60:  Topic 61:  Topic 62:  Topic 63:  Topic 64:  Topic 65:  Topic 66:  Topic 67:  Topic 68:  Topic 69:  Topic 70:  Topic 71:  Topic 72:  Topic 73:  Topic 74:  Topic 75:  Topic 76:  Topic 77:  Topic 78:  Topic 79:  Topic 80:  Topic 81:  Topic 82:  Topic 83:  Topic 84:  Topic 85:  Topic 86:  Topic 87:  Topic 88:  Topic 89:  Topic 90:  Topic 91

# Фильтрация слов

In [79]:
with open('lurk_parsed_full_total_sort.vv', 'r') as f:
    lines = f.read().split('\n')

print 'Lines: {}'.format(len(lines))

f = open('lurk_parsed_full_total_sort_3plus.vv', 'w')

topwords = [
'1', 
'11', 
'0', 
'00',
'01', 
'011', 
'10',
'это', 
'тот', 
'том', 
'год',
'свой', 
'такой', 
'ещ', 
'самый',
'также', 
'один', 
'весь', 
'время',
'чо', 
'этот', 
'который', 
'наш',
'вс', 
'человек', 
'быть', 
'мочь',
'стать', 
'метр', 
'другой', 
'дело',
'просто', 
'очень', 
'иметь', 
'два',
'являться', 
'случай', 
'the', 
'часть', 
'число',
'просто', 
'очень', 
'иметь', 
'два',
'страна',
'русский',
'мир',
'жизнь',
'слово',
'сабж',
'сайт',
'игрок',
'статья',
'место',
'война',
'советский',
'друг',
'день',
'персонаж',
'вид',
'либо',
'россия',
'фильм',
'сделать',
'говорить',
'город',
'какой',
'новый',
'вообще',
'делать',
'хотя',
]

i = 0
for line in lines:
    parts = line.split('|')
    if len(parts) < 2:
        continue

    words = ''
    cnt = 0
    for w in parts[1].split():
        splitted = w.split(':')
        if len(splitted) != 2:
            continue
        if int(splitted[1]) <= 2:
            continue

        if len(splitted[0]) < 3:
            continue

        if splitted[0] in topwords:
            continue

        cnt += 1
        words += ' ' + w
    i += 1
    if cnt >= 5:
        if i % 100 == 0:
            print i,
#         print parts[0], i,
        f.write(parts[0] + ' |' + words + '\n')

f.close()

Lines: 6218
200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2400 2600 2700 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4800 5000 5100 5200 5400 5700 5900 6000 6200
